In [70]:
from __future__ import division
from sklearn.preprocessing import CategoricalEncoder
from sklearn.preprocessing import LabelEncoder

import torchvision.models as models
from multiprocessing import Pool
import time
import numpy as np
import pandas as pd
import datetime as dt
import os.path as dpath
import matplotlib.pyplot as plt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import pdb
from pathlib import Path
import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F

from IPython.display import FileLink, FileLinks


%matplotlib inline

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [71]:
path = "/home/edwin/Datasets/competitions/avito-demand-prediction/"
PATH = Path(path)

In [72]:
TRN_CSV = pd.read_csv(PATH/'train.csv')
TRN_CSV_COPY = TRN_CSV.copy()

TEST_CSV = pd.read_csv(PATH/'test.csv')
TEST_CSV_COPY = TEST_CSV.copy()

In [77]:
TRN_CSV_COPY.deal_probability.max()

1.0

In [79]:
n = len(TRN_CSV_COPY); n

1503424

In [5]:
n_test = len(TEST_CSV_COPY); n_test

508438

In [80]:
idxs = get_cv_idxs(n, val_pct=15000/n)

In [8]:
# ?? get_cv_idxs

In [9]:
len(idxs)

15000

In [81]:
trn_sample = TRN_CSV_COPY.iloc[idxs]

In [83]:
trn_sample['image_top_1']

143309      256.0
771572      796.0
1378472     655.0
846158       91.0
84567        81.0
984506      404.0
1276450    2942.0
1063503    2266.0
170088     1334.0
1334881     558.0
1492727    1787.0
1435330     777.0
1219515    1002.0
308138      772.0
833263        NaN
1466866    1036.0
954186     2795.0
1467040    2939.0
841485      107.0
387023     2897.0
738740      595.0
1042955     501.0
68461       824.0
996059      690.0
15498       785.0
1354602    1102.0
642727       94.0
1099928     527.0
395782      106.0
966568     1990.0
            ...  
653352       85.0
1472013     489.0
1201582    2065.0
627956       44.0
582477     2737.0
461125      811.0
1427767       NaN
994475     1429.0
1186166     796.0
101465     2264.0
1028107    1377.0
28471      1993.0
1305140    1320.0
342052     1015.0
1040721    2648.0
954445     1180.0
381584     1106.0
1159959     399.0
615307       94.0
256011      675.0
245066      781.0
791408       94.0
123135     1272.0
515004       59.0
743964    

In [149]:
trn_sample["description"].fillna("").apply(lambda x: len(x.split()))

143309      13
771572     308
1378472      4
846158     143
84567       11
984506      40
1276450     42
1063503     19
170088      37
1334881     18
1492727     81
1435330     28
1219515    259
308138      15
833263      19
1466866      5
954186       8
1467040     45
841485       0
387023      11
738740       7
1042955     22
68461       18
996059     110
15498      218
1354602     28
642727      21
1099928      7
395782       7
966568      35
          ... 
653352       6
1472013      4
1201582     19
627956       8
582477       4
461125       7
1427767     32
994475      30
1186166      1
101465      13
1028107     48
28471        7
1305140     40
342052      16
1040721     14
954445      30
381584      42
1159959     45
615307       0
256011      17
245066       7
791408       0
123135      11
515004       0
743964      18
1307669      6
668633      18
88877       84
1058179     21
808649       0
Name: description, Length: 15000, dtype: int64

In [78]:
# from graph correlated -> deal_probability, description len, title_len 

In [175]:
trn_sample.sample()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
1330557,dcf7175358e5,4992d8c7fc8c,Свердловская область,Екатеринбург,Услуги,Предложение услуг,"Транспорт, перевозки",Аренда авто,NaN,Нексия в Аренду,В Аренду под выкуп Нексия в отличном состоянии...,750.0,1,2017-03-26,Private,4d8673009b01b11443ac8ae468f79eaa054a3f7344e097...,2264.0,1.0


In [200]:
class AvitoDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.n = len(df)
        
        self.df["description_len"] = self.df["description"].fillna("").apply(lambda x: len(x.split()))
        self.df["title_len"] = self.df["title"].fillna("").apply(lambda x: len(x.split()))
        
        self.cont_vars = ['price', 'image_top_1', "description_len", "title_len"]
        self.cat_vars = ['region', 'city', 'parent_category_name', 'category_name']
        self.y_var = 'deal_probability'
        
        self.df_cont = df[self.cont_vars].fillna(0)
        self.df_cat = df[self.cat_vars]
        
        for c in self.cont_vars:
            self.df_cont[c] = self.df_cont[c].astype("float32")
        
        for c in self.cat_vars:
            self.df_cat[c] = self.df_cat[c].fillna("").astype('category').cat.as_ordered()
            
        self.cat_sz = [(c, len(self.df_cat[c].cat.categories) + 1) for c in self.cat_vars]
        self.emb_szs = [(c, min(50, (c+1)//2)) for _, c in self.cat_sz]
        
        for n, c in self.df_cat.items():
            self.df_cat[n] = self.df_cat[n].cat.codes+1
            
        self.conts = np.stack([c.values for n, c in self.df_cont.items()], 1).astype(np.float32)
        self.cats = np.stack([c.values for n, c in self.df_cat.items()], 1).astype(np.int64)
        
        
        if self.y_var in df:
            self.y = df['deal_probability'][:, None].astype(np.float32)
        else:
            self.y = np.zeros([self.n,1]).astype(np.float32)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x_cat = self.cats[idx]
        x_cont = self.conts[idx]
        y = self.y[idx]
        return [x_cat, x_cont, y]
    

In [13]:
n = len(TRN_CSV_COPY)

In [14]:
train_ratio = 0.75

In [15]:
train_size = int(n * train_ratio); train_size

1127568

In [16]:
val_idxs = list(range(train_size, n))

In [17]:
len(TRN_CSV_COPY.iloc[val_idxs])

375856

In [201]:
train_ds = AvitoDataset(TRN_CSV_COPY)
valid_ds = AvitoDataset(TRN_CSV_COPY.iloc[val_idxs])
test_ds = AvitoDataset(TEST_CSV_COPY)
n_cont = len(train_ds.cont_vars); n_cont
emb_szs = train_ds.emb_szs

/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

In [202]:
train_dl = DataLoader(train_ds, batch_size=64, num_workers=1, shuffle=False)
valid_dl = DataLoader(valid_ds, batch_size=64, num_workers=1, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=64, num_workers=1, shuffle=False)

In [203]:
class AvitoMixedModel(nn.Module):
    def __init__(self, emb_szs, n_cont):
        super().__init__()
        self.embs = nn.ModuleList([nn.Embedding(c,s) for c, s in emb_szs])
        self.n_emb = sum([e.embedding_dim for e in self.embs])
        
        self.lin_size = self.n_emb + n_cont
        
        self.bn = nn.BatchNorm1d(n_cont)
        self.fc1 = nn.Linear(self.lin_size, 100)
        self.fc2 = nn.Linear(100, 1)
        self.emb_drop = nn.Dropout(0.04)
        
    def forward(self, x_cat, x_cont):
        x = [e(v_cat[:,i]) for i, e in enumerate(self.embs)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)
        x2 = self.bn(x_cont)
        x = torch.cat([x, x2], 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = x*(1-0)
        x = x+0
        return x
        

In [204]:
model = AvitoMixedModel(emb_szs, n_cont)

In [205]:
model.cuda()

AvitoMixedModel(
  (embs): ModuleList(
    (0): Embedding(29, 15)
    (1): Embedding(1734, 50)
    (2): Embedding(10, 5)
    (3): Embedding(48, 24)
  )
  (bn): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True)
  (fc1): Linear(in_features=98, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
  (emb_drop): Dropout(p=0.04)
)

In [222]:
lr = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [ ]:
# RMSE Error: Need to calculate 1/n sum(y - y_hat)^2 

In [ ]:
# description
# image quality

In [ ]:
corr = train_df.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.set(style="white")

f, ax = plt.subplots(figsize=(10, 8))
cmap = sns.diverging_palette(30, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.1, cbar_kws={"shrink": .5});

In [224]:
num_epochs = 100
n_iter = 0
for epoch in range(num_epochs):    
    for i, data in enumerate(train_dl,0):
        x_cat, x_cont, y = data
        v_cat = Variable(x_cat.cuda()); v_cont = Variable(x_cont.cuda()); vy = Variable(y.cuda());

        optimizer.zero_grad()

        outputs = model(v_cat, v_cont)
        loss = criterion(outputs, vy)
        loss.backward()
        optimizer.step()
        n_iter += 1
        if i % 5000 == 0:
            errors = []
            for data in valid_dl:
                x_cat, x_cont, y = data
                v_cat = Variable(x_cat.cuda()); v_cont = Variable(x_cont.cuda()); vy = Variable(y.cuda());
                outputs = model(v_cat, v_cont)
                errors.append(((vy - outputs)*(vy-outputs)).data.cpu().numpy())
            errors = np.concatenate(errors).ravel() 
            rmse_error = errors.sum() / len(errors)
            print("epoch: {}, i:{}, loss:{}, rmse_error:{}".format(epoch,n_iter,loss.data[0],rmse_error))    
print("Finished Training")

epoch: 0, i:1, loss:0.06566165387630463, rmse_error:0.058804061595398235
epoch: 0, i:5001, loss:0.05211956053972244, rmse_error:0.05877158364839194
epoch: 0, i:10001, loss:0.06280845403671265, rmse_error:0.05878907497153165
epoch: 0, i:15001, loss:0.05791556462645531, rmse_error:0.05882994522004438
epoch: 0, i:20001, loss:0.045138392597436905, rmse_error:0.05876159083365704
epoch: 1, i:23492, loss:0.06510350108146667, rmse_error:0.05875792732123473
epoch: 1, i:28492, loss:0.05080758035182953, rmse_error:0.05878914252566143
epoch: 1, i:33492, loss:0.06299901008605957, rmse_error:0.058817406134264186
epoch: 1, i:38492, loss:0.05797632411122322, rmse_error:0.05886470961451992
epoch: 1, i:43492, loss:0.044603534042835236, rmse_error:0.058760063071029864
epoch: 2, i:46983, loss:0.06575149297714233, rmse_error:0.058764609983610744
epoch: 2, i:51983, loss:0.05205860733985901, rmse_error:0.05880469036845228
epoch: 2, i:56983, loss:0.06231585890054703, rmse_error:0.05884336770598314
epoch: 2, i

epoch: 21, i:498312, loss:0.052300140261650085, rmse_error:0.05893746021581404
epoch: 21, i:503312, loss:0.061405427753925323, rmse_error:0.05897150749721968
epoch: 21, i:508312, loss:0.05813663825392723, rmse_error:0.0590049052196852
epoch: 21, i:513312, loss:0.046030692756175995, rmse_error:0.05892444825112543
epoch: 22, i:516803, loss:0.06444722414016724, rmse_error:0.05893639493915223
epoch: 22, i:521803, loss:0.050745271146297455, rmse_error:0.058917287513369485
epoch: 22, i:526803, loss:0.061427392065525055, rmse_error:0.058949791442733386
epoch: 22, i:531803, loss:0.05713171139359474, rmse_error:0.05900322156291239
epoch: 22, i:536803, loss:0.0449400395154953, rmse_error:0.058910901049870165
epoch: 23, i:540294, loss:0.06643037497997284, rmse_error:0.058908640584758525
epoch: 23, i:545294, loss:0.05296679958701134, rmse_error:0.05891003323912616
epoch: 23, i:550294, loss:0.062280621379613876, rmse_error:0.058966591635160805
epoch: 23, i:555294, loss:0.058845072984695435, rmse_er

epoch: 42, i:991623, loss:0.052256595343351364, rmse_error:0.058997624963084264
epoch: 42, i:996623, loss:0.059921376407146454, rmse_error:0.059026943455405795
epoch: 42, i:1001623, loss:0.059073127806186676, rmse_error:0.059076169630124305
epoch: 42, i:1006623, loss:0.04581126198172569, rmse_error:0.059016747978281575
epoch: 43, i:1010114, loss:0.06459690630435944, rmse_error:0.05895344456221266
epoch: 43, i:1015114, loss:0.05171136185526848, rmse_error:0.058985605524456176
epoch: 43, i:1020114, loss:0.06345804780721664, rmse_error:0.05904489726451088
epoch: 43, i:1025114, loss:0.060494087636470795, rmse_error:0.05908475939754853
epoch: 43, i:1030114, loss:0.04561253637075424, rmse_error:0.058991378804316014
epoch: 44, i:1033605, loss:0.06477083265781403, rmse_error:0.05899505270968137
epoch: 44, i:1038605, loss:0.05166751891374588, rmse_error:0.05897686505935784
epoch: 44, i:1043605, loss:0.06167358160018921, rmse_error:0.05905054063258269
epoch: 44, i:1048605, loss:0.058142464607954

epoch: 63, i:1479934, loss:0.0641913041472435, rmse_error:0.05901710133834501
epoch: 63, i:1484934, loss:0.054151132702827454, rmse_error:0.05901721046424695
epoch: 63, i:1489934, loss:0.058713436126708984, rmse_error:0.059077946823384485
epoch: 63, i:1494934, loss:0.058860521763563156, rmse_error:0.059142881931511
epoch: 63, i:1499934, loss:0.04675396531820297, rmse_error:0.059046232757691774
epoch: 64, i:1503425, loss:0.06313800811767578, rmse_error:0.05902902724048572
epoch: 64, i:1508425, loss:0.052723467350006104, rmse_error:0.05904932465824678
epoch: 64, i:1513425, loss:0.060613542795181274, rmse_error:0.05907105110567611
epoch: 64, i:1518425, loss:0.057560309767723083, rmse_error:0.0591441914423343
epoch: 64, i:1523425, loss:0.04608059674501419, rmse_error:0.05905347144252054
epoch: 65, i:1526916, loss:0.0647163987159729, rmse_error:0.059036837537181794
epoch: 65, i:1531916, loss:0.05301694571971893, rmse_error:0.05904965723242412
epoch: 65, i:1536916, loss:0.06061680242419243, 

epoch: 83, i:1969754, loss:0.047583453357219696, rmse_error:0.05912548934133019
epoch: 84, i:1973245, loss:0.06427087634801865, rmse_error:0.05909462749696692
epoch: 84, i:1978245, loss:0.05201878771185875, rmse_error:0.059121898579509176
epoch: 84, i:1983245, loss:0.06008288264274597, rmse_error:0.05916157363957207
epoch: 84, i:1988245, loss:0.05968960374593735, rmse_error:0.059202236029223955
epoch: 84, i:1993245, loss:0.04700683802366257, rmse_error:0.05912286512321208
epoch: 85, i:1996736, loss:0.06412313133478165, rmse_error:0.059134011554624645
epoch: 85, i:2001736, loss:0.053460489958524704, rmse_error:0.05913659420097058
epoch: 85, i:2006736, loss:0.061209216713905334, rmse_error:0.05917532350321666
epoch: 85, i:2011736, loss:0.0571267232298851, rmse_error:0.05923450611736942
epoch: 85, i:2016736, loss:0.046647898852825165, rmse_error:0.05914133338299774
epoch: 86, i:2020227, loss:0.06505218148231506, rmse_error:0.05913643311035343
epoch: 86, i:2025227, loss:0.05148044973611832

In [223]:
outputs

Variable containing:
 0.1327
 0.1042
 0.0314
 0.0315
 0.0757
 0.0277
 0.3201
 0.0460
 0.1734
 0.0542
 0.0893
 0.1830
 0.2650
 0.2288
 0.2337
 0.0951
 0.2295
 0.1075
 0.0707
 0.0614
 0.0685
 0.0341
 0.4328
 0.1658
 0.1936
 0.0743
 0.0148
 0.0432
 0.0772
 0.0816
 0.0404
 0.1922
 0.0420
 0.2461
 0.1566
 0.2181
 0.1535
 0.0525
 0.0274
 0.0376
 0.1591
 0.0390
 0.2045
 0.0975
 0.1379
 0.1963
 0.1899
 0.0559
 0.0639
 0.0673
 0.0412
 0.0200
 0.0323
 0.1556
 0.2160
 0.0933
 0.0158
 0.3067
 0.0393
 0.0698
 0.4248
 0.3690
 0.0516
 0.0856
[torch.cuda.FloatTensor of size 64x1 (GPU 0)]

In [209]:
torch.save(model.state_dict(), 'weights_0521.pkl')

In [225]:
res = []
for i, data in enumerate(test_dl, 0):
    x_cat, x_cont, y = data
    v_cat = Variable(x_cat.cuda()); v_cont = Variable(x_cont.cuda()); vy = Variable(y.cuda());
    outputs = model(v_cat, v_cont)
    res.append(outputs.data.cpu().numpy())
predictions = np.concatenate(res).ravel()

In [226]:
NEW_SUBMIT_CSV = pd.DataFrame()
NEW_SUBMIT_CSV['item_id'] = TEST_CSV_COPY.item_id.values
NEW_SUBMIT_CSV['deal_probability'] = predictions
NEW_SUBMIT_CSV.to_csv('new_submission.csv', index=False)
FileLink('new_submission.csv')

/home/edwin/Developer/iMaterial/avito-demand-prediction/new_submission.csv